In [1]:
import yaml
import pandas as pd
import matplotlib.pyplot as plt
#lens_yaml = open("../../SLACS_directory/settings/Shajib2020_Lens.yml")
#lenses_yaml = yaml.load(lens_yaml, yaml.FullLoader)
from dolphin.analysis.output import Output
from lenstronomy.Util.param_util import ellipticity2phi_q
import numpy as np
import h5py


In [2]:
def stats_giver(chain):
    mean = np.median(chain)
    d1 = np.percentile(chain, 50.) - np.percentile(chain, 16.)
    d2 = np.percentile(chain, 84.) - np.percentile(chain, 50.)
    return mean, d1, d2

def angle_squeezer(x):
    if x<-90:
        return 180+x
    elif x>90:
        return x-180
    else:
        return x

#def ext_corrector(x):
#    if x>90:
#        return x-180
#    else:
#        return x    
    
    
def branch_corrector(x): #Correct for when the points are at -90,90 branch cuts
    if x<0:
        return x+180
    else:
        return x
        
def angle_stats_giver(chain):
    chain_corr = [angle_squeezer(x) for x in chain]
    chain_branch = [branch_corrector(x) for x in chain_corr ]    
    a1,a2,a3 = stats_giver(chain_corr) 
    b1,b2,b3 = stats_giver(chain_branch) 
    if (abs(a2)+abs(a3)) < (abs(b2)+abs(b3)):
        return a1,a2,a3
    else:  
        return angle_squeezer(b1),b2,b3        




In [3]:
def ingestor(obj):
    data_dict = {"Lens_name":obj}
    chain = output.get_reshaped_emcee_chain(obj, "dinos-i" ,16)
    params_mcmc = output.params_mcmc
    
    psiM_chain, qM_chain = ellipticity2phi_q(chain[:, -1, params_mcmc.index('e1_lens0')],chain[:, -1, params_mcmc.index('e2_lens0')]) 
    psiL_chain, qL_chain = ellipticity2phi_q(chain[:, -1, params_mcmc.index('e1_lens_light0')],chain[:, -1, params_mcmc.index('e2_lens_light0')]) 

    #Convert angles:
    psi_ext_chain = chain[:, -1, params_mcmc.index('psi_ext_lens1')]*180/np.pi
    psiM_chain = psiM_chain *180/np.pi
    psiL_chain = psiL_chain *180/np.pi
    
    data_dict['R_Ein'], data_dict['R_Ein_e1'], data_dict['R_Ein_e2']   = stats_giver(chain[:, -1, params_mcmc.index('theta_E_lens0')])
    data_dict['γ'], data_dict['γ_e1'], data_dict['γ_e2']               = stats_giver(chain[:, -1, params_mcmc.index('gamma_lens0')])
    data_dict['PA ext'], data_dict['PA ext_e1'],data_dict['PA ext_e2'] = angle_stats_giver(psi_ext_chain)
    data_dict['γ ext'], data_dict['γ ext_e1'], data_dict['γ ext_e2']   = stats_giver(chain[:, -1, params_mcmc.index('gamma_ext_lens1')])
    data_dict['PA m'], data_dict['PA m_e1'], data_dict['PA m_e2']      = angle_stats_giver(psiM_chain)
    data_dict['qm'],data_dict['qm_e1'],data_dict['qm_e2']              = stats_giver(qM_chain)
    data_dict['e1 m'], data_dict['e1 m_e1'],data_dict['e1 m_e2']       = stats_giver(chain[:, -1, params_mcmc.index('e1_lens0')])
    data_dict['e2 m'], data_dict['e2 m_e1'],data_dict['e2 m_e2']       = stats_giver(chain[:, -1, params_mcmc.index('e2_lens0')])
    data_dict['PA L'],data_dict['PA L_e1'],data_dict['PA L_e2']        = angle_stats_giver(psiL_chain)
    data_dict['qL'], data_dict['qL_e1'],data_dict['qL_e2']             = stats_giver(qL_chain)
    data_dict['e1 L'], data_dict['e1 L_e1'],data_dict['e1 L_e2']       = stats_giver(chain[:, -1, params_mcmc.index('e1_lens_light0')])
    data_dict['e2 L'], data_dict['e2 L_e1'],data_dict['e2 L_e2']       = stats_giver(chain[:, -1, params_mcmc.index('e2_lens_light0')])    
        
    return data_dict

In [5]:
output = Output('../2_dolphin_modelling')
ingestor('SL2SJ1359+5535')

{'Lens_name': 'SL2SJ1359+5535',
 'R_Ein': 1.1065253826023058,
 'R_Ein_e1': 0.013208843065832365,
 'R_Ein_e2': 0.013524900263527373,
 'γ': 1.9510064727008452,
 'γ_e1': 0.15528847954890534,
 'γ_e2': 0.15205686446069722,
 'PA ext': -82.07156912917989,
 'PA ext_e1': 45.45000696101083,
 'PA ext_e2': 36.66857902986047,
 'γ ext': 0.003725562826294616,
 'γ ext_e1': 0.002344522130953911,
 'γ ext_e2': 0.004197204690525223,
 'PA m': 34.73079706470444,
 'PA m_e1': 0.9696183799718696,
 'PA m_e2': 1.2522436733284295,
 'qm': 0.6851752529809387,
 'qm_e1': 0.12139106451175807,
 'qm_e2': 0.07925493615024126,
 'e1 m': 0.06614221220376762,
 'e1 m_e1': 0.02022658083536187,
 'e1 m_e2': 0.02862538523280317,
 'e2 m': 0.17398968928746106,
 'e2 m_e1': 0.04933583754219008,
 'e2 m_e2': 0.08735881783736849,
 'PA L': 46.22324218918226,
 'PA L_e1': 4.359839425896745,
 'PA L_e2': 2.925843795287463,
 'qL': 0.6797251416657444,
 'qL_e1': 0.08421181314428228,
 'qL_e2': 0.07529182745064333,
 'e1 L': -0.0071621855993617995